In [20]:
# You should only need to run this cell once when you (re)start the kernel. Thereafter, includet should import any changes.
using Pkg
Pkg.activate("..") # change this to your package-install location

using MLDatasets: MNIST
using ImageCore
using Flux: onehotbatch, onecold
using Metrics
using BenchmarkTools

using CUDA
CUDA.allowscalar(false)

using Revise
includet("activations_and_losses.jl")
includet("dense_network_model.jl")
includet("dense_network_training.jl")

  Activating project at `~/Workspace/CSC381`


In [4]:
train_x, train_y = MNIST.traindata()
test_x,  test_y  = MNIST.testdata()

train_set_size = size(train_x)[end]
test_set_size = size(test_x)[end]
image_dimensions = size(train_x)[1:end-1]

println(train_set_size, " points in the training set")
println(test_set_size, " points in the testing set")
println("image inputs have dimension ", image_dimensions)

60000 points in the training set
10000 points in the testing set
image inputs have dimension (28, 28)


In [5]:
image_ID = rand(1:train_set_size)
println("image #", image_ID, ", label = ", train_y[image_ID])
MNIST.convert2image(train_x[:,:,image_ID])

image #33079, label = 0


In [15]:
num_inputs = size(train_x,1) * size(train_x,2)
num_labels = length(unique(train_y))
hidden_sizes = [100,100,50,10]
hidden_activations = [ReLU_activation, ReLU_activation, ReLU_activation, ReLU_activation]
output_activation = softmax_activation
nn1 = DenseNetworkCPU(num_inputs, num_labels, hidden_sizes; hidden_activations=hidden_activations, output_activation=output_activation);
nn2 = DenseNetworkGPU(num_inputs, num_labels, hidden_sizes; hidden_activations=hidden_activations, output_activation=output_activation);

In [ ]:
cu(Array{Float32}(reshape(permutedims(train_x, [3,1,2]), 60000, 28*28)))

In [ ]:
cu(Array{Float32}(onehotbatch(train_y, 0:9)'))

In [16]:
inputs = Array{Float32}(reshape(permutedims(train_x, [3,1,2]), train_set_size, num_inputs))
GPU_inputs = cu(inputs)
targets = onehotbatch(train_y, 0:9)'
GPU_targets = cu(Array{Float32}(targets))
println("input shape: ", size(inputs))
println("target shape: ", size(targets))

input shape: (60000, 784)
target shape: (60000, 10)


In [28]:
epochs = 1
batch_size = 1000000

@benchmark CUDA.@sync train!(nn1, inputs, targets, 0.1, epochs, batch_size; verbose=true)

epoch #1 ... average loss = 2.2111588
epoch #1 ... average loss = 2.0372195
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN


BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.802 s … 5.694 s  ┊ GC (min … max): 0.29% … 0.16%
 Time  (median):     4.248 s            ┊ GC (median):    0.20%
 Time  (mean ± σ):   4.248 s ± 2.045 s  ┊ GC (mean ± σ):  0.20% ± 0.09%

  █                                                     █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.8 s         Histogram: frequency by time       5.69 s <

 Memory estimate: 748.24 MiB, allocs estimate: 505.

In [29]:
@benchmark CUDA.@sync train!(nn2, GPU_inputs, GPU_targets, 0.1, epochs, batch_size; verbose=true)

epoch #1 ... average loss = 2.2608552
epoch #1 ... average loss = 2.2526152
epoch #1 ... average loss = 2.0883303
epoch #1 ... average loss = 21.581697
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN
epoch #1 ... average loss = NaN


BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  274.125 ms … 825.842 ms  ┊ GC (min … max): 1.70% … 0.65%
 Time  (median):     465.639 ms               ┊ GC (median):    1.13%
 Time  (mean ± σ):   485.937 ms ± 149.229 ms  ┊ GC (mean ± σ):  1.25% ± 0.53%

  ▁       ▁     █  ▁   ▁  ▁ ▁ ▁           ▁                   ▁  
  █▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁█▁▁▁█▁▁█▁█▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  274 ms           Histogram: frequency by time          826 ms <

 Memory estimate: 411.32 MiB, allocs estimate: 4202.

In [13]:
function test_accuracy(y_pred, real)
    CUDA.allowscalar(true)
    correct = 0
    incorrect = 0
    for i in 1:size(y_pred, 1)
        test_pred = y_pred[i, :]
        test_real = real[i, :]
        if test_pred[test_real][1] > 0.9
            correct += 1
        else
            incorrect +=1
        end 
    end
    percent_accurate = correct / (correct + incorrect)
end

test_accuracy (generic function with 1 method)

In [1]:
test_inputs = Array{Float32}(reshape(permutedims(test_x, [3,1,2]), test_set_size, num_inputs))

LoadError: syntax: break or continue outside loop

In [10]:
targets = onehotbatch(test_y, 0:9)'

10000×10 adjoint(OneHotMatrix(::Vector{UInt32})) with eltype Bool:
 0  0  0  0  0  0  0  1  0  0
 0  0  1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  1
 ⋮              ⋮           
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  1  0  0  0

In [11]:
y_pred = predict(nn1, test_inputs)
test_accuracy(y_pred, targets)

0.7394

In [14]:
y_pred = predict(nn2, test_inputs)
test_accuracy(y_pred, targets)

0.7418

In [21]:
test_real=targets[1, :]

10-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0

In [22]:
test_pred=y_pred[1,:]

10-element Vector{Float32}:
 0.0028972477
 0.00049888354
 2.9027315f-5
 0.0024205318
 0.0014580422
 0.009566526
 6.5120956f-5
 0.95879036
 0.00051547517
 0.023758825

In [18]:
test_real == test_pred[test_real == 1]

LoadError: ArgumentError: invalid index: false of type Bool

In [27]:
test_pred[test_real][1] > 0.9

true

In [51]:
Metrics.f_beta_score(y_pred, targets)

LoadError: InterruptException:

In [13]:
y_pred[1,:]

10-element Vector{Float32}:
 0.0031984802
 0.00054789794
 0.00074891
 0.0011589474
 0.00033770085
 5.5628345f-5
 0.0003416134
 0.98684746
 2.5765994f-5
 0.006737474

In [60]:
test = copy(y_pred[1,:])

10-element Vector{Float32}:
 0.0031984802
 0.00054789794
 0.00074891
 0.0011589474
 0.00033770085
 5.5628345f-5
 0.0003416134
 0.98684746
 2.5765994f-5
 0.006737474

In [1]:
y_pred[1,:]

LoadError: UndefVarError: y_pred not defined

In [61]:
test[test .> 0.8] .= 1

1-element view(::Vector{Float32}, [8]) with eltype Float32:
 1.0

In [63]:
test[test .< 1] .= 0

9-element view(::Vector{Float32}, [1, 2, 3, 4, 5, 6, 7, 9, 10]) with eltype Float32:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [68]:
test == targets[1,:]

true

In [ ]:
nn1.output_layer.activation == softmax_activation

In [27]:
@benchmark CUDA.@sync zeros(5, 5)

BenchmarkTools.Trial: 10000 samples with 84 evaluations.
 Range (min … max):  866.405 ns …  1.433 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     914.036 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   920.341 ns ± 29.313 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

              ▃██▄▃▄▆▃  ▁                                       
  ▂▁▁▁▁▁▂▁▂▁▂▅████████▇▇█▇▅▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  866 ns          Histogram: frequency by time         1.03 μs <

 Memory estimate: 256 bytes, allocs estimate: 1.